## What is Numba?

Numba is a just-in-time, type-specializing, function compiler for accelerating numerically-focused Python. That's a long list, so let's break down those terms:

  1. function compiler: Numba compiles Python functions, not entire applications, and not parts of functions. Numba does not replace your Python interpreter, but is just another Python module that can turn a function into a (usually) faster function.
  1. type-specializing: Numba speeds up your function by generating a specialized implementation for the specific data types you are using. Python functions are designed to operate on generic data types, which makes them very flexible, but also very slow. In practice, you only will call a function with a small number of argument types, so Numba will generate a fast implementation for each set of types.
  1. just-in-time: Numba translates functions when they are first called. This ensures the compiler knows what argument types you will be using. This also allows Numba to be used interactively in a Jupyter notebook just as easily as a traditional application
  1. numerically-focused: Currently, Numba is focused on numerical data types, like int, float, and complex. There is very limited string processing support, and many string use cases are not going to work well on the GPU. To get best results with Numba, you will likely be using NumPy arrays.

In [ ]:
import numba

In [ ]:
def pyfunc(x):
    return 3.0*x**2 + 2.0*x + 1.0

pyfunc(3.14)

In [ ]:
# a compiled function that has no Python objects in the body,
# but it can be used in Python because it interprets Python on the way in
@numba.jit(nopython=True)
def fastfunc(x):
    return 3.0*x**2 + 2.0*x + 1.0

fastfunc(3.14)

In [ ]:
# have to provide a signature: "double (double, void*)"
sig = numba.types.double(numba.types.double,
                         numba.types.CPointer(numba.types.void))
# a pure C function that doesn't interpret Python arguments
@numba.cfunc(sig, nopython=True)
def cfunc(x, params):
    return 3.0*x**2 + 2.0*x + 1.0

cfunc(3.14)   # should raise an error

In [ ]:
# we get a function pointer instead
cfunc.address

In [ ]:
# just to verify that this pointer works, get ctypes to use it
import ctypes

func_type = ctypes.CFUNCTYPE(ctypes.c_double, ctypes.c_double, ctypes.POINTER(None))

func_type(cfunc.address)(3.14, None)